# Mounting the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing the required libraries

In [ ]:
import os
import math
import ast
import glob
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
!pip install autocorrect
from autocorrect import Speller
from collections import OrderedDict 
from pathlib import Path
import shutil
import spacy 
import re
nltk.download('wordnet')
nltk.download('stopwords')
!pip install contractions
!pip install spacy
!pip install textblob
import contractions
from textblob import TextBlob, Word
import pandas as pd
import numpy as np

     |████████████████████████████████| 624kB 4.5MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.4.0-cp37-none-any.whl size=621775 sha256=b2665def417998e5e306f5f0b2c50c9197b2316737c908671862c07828975069
  Stored in directory: /root/.cache/pip/wheels/1d/b0/d4/b941891ad0f8d8847be03583e21e68ed4732d763c71a6c0943
Successfully built autocorrect
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 327kB 5.5MB/s 
     |████████████████████████████████| 266kB 7.9MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85398 sha256=be4f2f603710a676f6a920554c6db63f3dcd04080ca3d2c210af0216c003f18c
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


# TFIDF

# Preprocessing

In [ ]:
documentsDict = {}

def cleaning(filepath,filename,flag):
  if(flag==1):
    f = open(filepath, 'r', errors = 'ignore',encoding='cp1250').read().strip()
    clean_S = contractions.fix(f)
    clean_S=re.sub("([\-]{1,2})", " ",clean_S)
    clean_s=re.sub("[\,]", " ",clean_S)
    clean_S = re.sub(r'[\"\']*',"", clean_S)  #junk symbols
    clean_S=re.sub("([\.\!]{2})", " ",clean_S)  #removing punctuations
    clean_S = re.sub('\s+', ' ', clean_S)    
    clean_S=re.sub("[^a-zA-Z\s\n]", " ",clean_S)  #used to eliminate non-ascii and numeric symbols 
    clean_S = re.sub(r"(.)\1{2}","",clean_S)  #for elongated words
    clean_S = clean_S.lower()
    documentsDict[filename]=clean_S

  elif(flag==0):
    filepath = contractions.fix(filepath)
    filepath=re.sub("([\-]{1,2})", " ",filepath)
    filepath=re.sub("[0-9]", "",filepath)
    filepath=re.sub("[\,]", " ",filepath)
    filepath = re.sub(r'[\"\']*',"", filepath)
    # filepath = re.sub(r"(.)\1{2}","",filepath)  #for elongated words
    filepath=re.sub("([\.\!]{2,})", " ",filepath)
    filepath = re.sub('\s+', ' ', filepath)
    filepath=re.sub("[^a-zA-Z\s\n]", " ",filepath)
    filepath = re.sub(r"(.)\1{2}","",filepath)  #for elongated words
    filepath = filepath.lower()
    return filepath

In [ ]:
def remove_stopword(flag,query):
  stopwordsList = stopwords.words('english')
  if(flag==1):
    for docname in documentsDict:
        for word in documentsDict[docname]:
            if word in stopwordsList:
                documentsDict[docname].remove(word)
  elif(flag==0):
      q=[]
      for word in query:
            if word not in stopwordsList:
              q.append(word)
      query=q.copy()
      return query

# Load data

In [ ]:
tempdoc={}
for docid in range(470):
  temp={}
  for col in file2:
     if docid in file2.iloc[1][col]:
       temp[col]=file2.iloc[1][col][docid]
  tempdoc[docid] = temp  
f = open('tempdoc.txt', 'w')
f.write(str(tempdoc))
f.close()

In [ ]:
def create_docFreq(file1):
  docFreq={}
  stopwordsList = stopwords.words('english')
  for column in file1.columns:
    if column not in stopwordsList:
      docFreq[column] = file1.iloc[0][column]
  return docFreq
doc_frequency = create_docFreq(file1)
f= open("/content/drive/MyDrive/IR ASSIGNMENT 2/docFrequency.txt","w")
f.write(str(doc_frequency))
f.close()

In [ ]:
def count_dict(file1):#vocab
  termFrequency = {}
  stopwordsList = stopwords.words('english')
  for column in file1.columns:
    if column not in stopwordsList:
      xa = ast.literal_eval(file1.iloc[1][column])
      listoflist = list(xa.values())
      counter = 0
      for i in range(0,len(listoflist)):
        counter =counter+len(listoflist[i])
      termFrequency[column] = counter
  return termFrequency
dict1 = count_dict(file1)
f= open("/content/drive/MyDrive/IR/vocab.txt","w")
f.write(str(dict1))
f.close()

In [ ]:
def loadStoredFiles():
  f = open('/content/drive/MyDrive/IR/documentsFilename.txt', 'r')
  documentfilename = f.read()
  f.close()
  documentfilename = documentfilename.split(",")
  file1 = pd.read_csv("/content/drive/MyDrive/IR/PostingsFinal.csv" )
  file1 = file1.drop(file1.columns[0],axis=1)
  file2 = pd.read_csv("/content/drive/MyDrive/IR/FrequencyOfWord.csv" )
  file2 = file2.drop(file2.columns[0],axis=1)
  for col in file2:
     file2.iloc[1][col] = ast.literal_eval(file2.iloc[1][col])
  f = open('/content/drive/MyDrive/IR/tempdoc.txt', 'r')
  tempdoc = f.read()
  f.close()
  tempdoc = ast.literal_eval(tempdoc)
  f = open('/content/drive/MyDrive/IR/docFrequency.txt', 'r')
  docFrequency = f.read()
  f.close()
  docFrequency = ast.literal_eval(docFrequency)
  f = open('/content/drive/MyDrive/IR/vocab.txt', 'r')
  dict1 = f.read()
  f.close()
  dict1 = ast.literal_eval(dict1)
  return dict1,tempdoc,docFrequency,file1,file2,documentfilename

In [ ]:
def query_count(query_token,dict1):
    list1 = dict.fromkeys(query_token, 0)
    for word in query_token:
      if (word in dict1):
         list1[word] += 1
    return list1

# Compute TF

In [ ]:
def find_binary(dict1,dict2):
    tfDict = dict.fromkeys(dict1,0)
    for word,count in dict1.items():
      if word in dict2:
        count=dict2[word]
        if count >0 :
          tfDict[word] = 1
        else:
          tfDict[word] = 0
    return tfDict   
def forBinary(dict1,dict2,tempdoc):
  tf1 = find_binary(dict1,dict2)
  tf_doc = dict.fromkeys(tempdoc,0)
  for word,c in tempdoc.items():
    tf_doc[word]=find_binary(dict1,c)
  return tf1,tf_doc

In [ ]:
def find_rawCount(dict1,dict2):
    tfDict = dict.fromkeys(dict1,0)
    for word,count in dict1.items():
      if word in dict2:
        count=dict2[word]
        tfDict[word] = count
    return tfDict
def forRawCount(dict1,dict2,tempdoc):
  tf1 = find_rawCount(dict1,dict2)
  tf_doc = dict.fromkeys(tempdoc,0)
  for word,c in tempdoc.items():
    tf_doc[word]=find_rawCount(dict1,c)
  return tf1,tf_doc

In [ ]:
def findTF(wordDict, dict1):
    dict1Count = len(dict1)
    tfDict = dict.fromkeys(dict1,0)
    for word,count in dict1.items():
      if word in wordDict:
        count=wordDict[word]
        tfDict[word] = count / float(dict1Count)
    return tfDict
def forTF(dict1,dict2,tempdoc):
  tf1 = findTF(dict2, dict1)
  tf_doc = dict.fromkeys(tempdoc,0)
  for word,c in tempdoc.items():
    tf_doc[word]=findTF(c,dict1)
  return tf1,tf_doc

In [ ]:
def findlogTF(wordDict, dict1):
    tfDict = dict.fromkeys(dict1,0)
    for word,count in dict1.items():
          tfDict[word] = 1 + np.log(count)
    return tfDict
def forLogTF(dict1,dict2,tempdoc):
  tf1 = findlogTF(dict2, dict1)
  tf_doc = dict.fromkeys(tempdoc,0)
  for word,c in tempdoc.items():
    tf_doc[word]=findlogTF(c,dict1)
  return tf1,tf_doc

In [ ]:
def finddoublenormTF(wordDict, dict1):
    max=0
    for word,count in dict1.items():
      if max<count:
        max=count
    tfDict = dict.fromkeys(dict1,0)
    for word,count in dict1.items():
        tfDict[word] = 0.5+((0.5*(count)/max))
    return tfDict
def forNorm(dict1,dict2,tempdoc):
  tf1 = finddoublenormTF(dict2, dict1)
  tf_doc = dict.fromkeys(tempdoc,0)
  for word,c in tempdoc.items():
    tf_doc[word]=finddoublenormTF(c,dict1)
  return tf1,tf_doc

# Compute IDF

In [ ]:
def findIDF(doc,dict1,doc_frequency):
  idfDict = dict.fromkeys(dict1,0)
  for word,c in dict1.items():  
    if word in doc:
      if (word in doc_frequency):
          count1 = doc_frequency[word]
          # print(count1)
      idfDict[word] = np.log(467 / (float(count1)+1))
  return idfDict
def forIDF(dict1,dict2,tempdoc,docFrequency):
  idfs = findIDF(dict2,dict1,docFrequency)
  idf_doc = dict.fromkeys(tempdoc,0)
  for word,c in tempdoc.items():
    idf_doc[word]=findIDF(c,dict1,docFrequency)
  return idfs,idf_doc

# Compute TF-IDF

In [ ]:
def findTFIDF(dicttf, idfs):
  tfidf = {}
  for word, i in dicttf.items():
      tfidf[word] = i * idfs[word]
  return tfidf
def forTFIDF(tf1,idfs,tf_doc,idf_doc):
  tfidf1 = findTFIDF(tf1, idfs)
  df = pd.DataFrame([tfidf1])
  tfidfdict = {}
  for docid in range(0,467):
    temp = {}
    for word in tf_doc[0]:
      temp[word] = tf_doc[docid][word]*idf_doc[docid][word]
    tfidfdict[docid] = temp
  return tfidf1,tfidfdict

# Compute Scores

In [ ]:
def getRank(doc,query):
  score=dict.fromkeys(doc,0)
  for i in range(467):
    sum=0
    for c in query:
      sum+=doc[i][c]*query[c]
    score[i]=sum
  rankDoc = sorted(score, key=score.get, reverse=True)
  return rankDoc,score

In [ ]:
dict1,tempdoc,docFrequency,file1,file2,documentfilename = loadStoredFiles()

In [ ]:
def callingFun(choice,query,dict1,tempdoc,docFrequency,file1,file2,documentfilename):
  query = cleaning(query," ",0)
  query=query.split()
  query = remove_stopword(0,query)
  dict2 = query_count(query,dict1) 
  if choice == 'Binary':
    tf1,tf_doc = forBinary(dict1,dict2,tempdoc)
  elif choice == 'Raw count':
    tf1,tf_doc = forRawCount(dict1,dict2,tempdoc)
  elif choice == 'Term Frequency':
    tf1,tf_doc = forTF(dict1,dict2,tempdoc)
  elif choice == 'Log Normalization':
    tf1,tf_doc = forLogTF(dict1,dict2,tempdoc)
  elif choice == 'Double Normalization':
    tf1,tf_doc = forNorm(dict1,dict2,tempdoc)
  idfs,idf_doc = forIDF(dict1,dict2,tempdoc,docFrequency)
  tfidf1,tfidfdict = forTFIDF(tf1,idfs,tf_doc,idf_doc)
  docranked , scores =getRank(tfidfdict,tfidf1)
  return documentfilename, docranked , scores

# Give Query and choice of Weighting Scheme and Get scores

In [ ]:
query = "THE CRAB AND THE HERON"
choice='Double Normalization'
print('-------------------------------'+choice+'-------------------------------------')
documentfilename,docranked , scores = callingFun(choice,query,dict1,tempdoc,docFrequency,file1,file2,documentfilename)
for i in range(0,5):
  print("Document Name "+ str(i+1) + " : "+ str(documentfilename[docranked[i]]) +"    with TFIDF Scores :   "+ str(scores[docranked[i]]) )
choice='Log Normalization'
print('---------------------------------'+choice+'---------------------------------------')
documentfilename,docranked , scores = callingFun(choice,query,dict1,tempdoc,docFrequency,file1,file2,documentfilename)
for i in range(0,5):
  print("Document Name "+ str(i+1) + " : "+ str(documentfilename[docranked[i]]) +"    with TFIDF Scores :   "+ str(scores[docranked[i]]) )
choice='Term Frequency'
print('-------------------------------'+choice+'-------------------------------------')
documentfilename,docranked , scores = callingFun(choice,query,dict1,tempdoc,docFrequency,file1,file2,documentfilename)
for i in range(0,5):
  print("Document Name "+ str(i+1) + " : "+ str(documentfilename[docranked[i]]) +"    with TFIDF Scores :   "+ str(scores[docranked[i]]) )
choice='Raw count'
print('-----------------------------------'+choice+'-----------------------------------------')
documentfilename,docranked , scores = callingFun(choice,query,dict1,tempdoc,docFrequency,file1,file2,documentfilename)
for i in range(0,5):
  print("Document Name "+ str(i+1) + " : "+ str(documentfilename[docranked[i]]) +"    with TFIDF Scores :   "+ str(scores[docranked[i]]) )
choice='Binary'
print('-------------------------------------'+choice+'-----------------------------------------')
documentfilename,docranked , scores = callingFun(choice,query,dict1,tempdoc,docFrequency,file1,file2,documentfilename)
for i in range(0,5):
  print("Document Name "+ str(i+1) + " : "+ str(documentfilename[docranked[i]]) +"    with TFIDF Scores :   "+ str(scores[docranked[i]]) )

-------------------------------Double Normalization-------------------------------------
Document Name 1 :  'aesop11.txt'    with TFIDF Scores :   10.549035745704728
Document Name 2 :  'crabhern.txt'    with TFIDF Scores :   10.549035745704728
Document Name 3 :  'aesopa10.txt'    with TFIDF Scores :   4.162853488120446
Document Name 4 :  'fgoose.txt'    with TFIDF Scores :   4.162853488120446
Document Name 5 :  'long1-3.txt'    with TFIDF Scores :   4.162853488120446
-------------------------------Log Normalization-------------------------------------
Document Name 1 :  'aesop11.txt'    with TFIDF Scores :   609.6416903418934
Document Name 2 :  'crabhern.txt'    with TFIDF Scores :   609.6416903418934
Document Name 3 :  'aesopa10.txt'    with TFIDF Scores :   315.463840318558
Document Name 4 :  'fgoose.txt'    with TFIDF Scores :   315.463840318558
Document Name 5 :  'long1-3.txt'    with TFIDF Scores :   315.463840318558
-------------------------------Term Frequency-------------------

# top 10

In [ ]:
for i in range(0,10):
  print("Document Name "+ str(i+1) + " : "+ str(documentfilename[docranked[i]]) +"    with TFIDF Scores :   "+ str(scores[docranked[i]]) )

Document Name 1 :  'aesop11.txt'    with TFIDF Scores :   42.019022293601225
Document Name 2 :  'crabhern.txt'    with TFIDF Scores :   42.019022293601225
Document Name 3 :  'aesopa10.txt'    with TFIDF Scores :   16.539575694462723
Document Name 4 :  'fgoose.txt'    with TFIDF Scores :   16.539575694462723
Document Name 5 :  'long1-3.txt'    with TFIDF Scores :   16.539575694462723
Document Name 6 :  'missing.txt'    with TFIDF Scores :   16.539575694462723
Document Name 7 :  'timem.hac'    with TFIDF Scores :   16.539575694462723
Document Name 8 : ['13chil.txt'    with TFIDF Scores :   0.0
Document Name 9 :  'yukon.txt'    with TFIDF Scores :   0.0
Document Name 10 :  '6napolen.txt'    with TFIDF Scores :   0.0
